In [ ]:
import random
import networkx as nx
from copy import deepcopy
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
full_attack_graph = deepcopy(attack_graph)

### Assumptions

For now I assume the following
- target nodes can NOT be dropped. The defender is aware of all critical assets in his system
- Entry nodes can be dropped. The defender might not be aware of all entry points into the graph
- Intermediate nodes can be dropped.

When Plotting:
- If entry node (green) has been dropped I will colour the "new" entry nodes green instead
- If some intermediate node has been dropped and a path now ends in a "dead end", this will not be turned red, as the defender still is aware that this is not a target node.

In [ ]:
def create_defender_subgraph(graph, drop_percentage=0.2):
    """
    Creates a subgraph for the defender by removing a percentage of non-target nodes.
    
    Args:
        graph: NetworkX graph
        drop_percentage: Percentage of non-target nodes to drop (default: 0.2)
        
    Returns:
        NetworkX subgraph with random nodes removed
    """
    
    # Make a deep copy to avoid modifying the original
    sub_graph = deepcopy(graph)
    
    # Identify target nodes (nodes with no outgoing edges)
    target_nodes = []
    for n, d in sub_graph.out_degree():
        if d == 0:
            target_nodes.append(n)
    
    print(f"Identified {len(target_nodes)} target nodes: {target_nodes}")
    
    # Create list of non-target nodes that can be dropped
    droppable_nodes = []
    for n in sub_graph.nodes():
        if n not in target_nodes:
            droppable_nodes.append(n)
    
    # Calculate how many nodes to drop
    num_to_drop = max(1, int(len(droppable_nodes) * drop_percentage))
    
    # Randomly select nodes to drop
    nodes_to_drop = random.sample(droppable_nodes, num_to_drop)
    print(f"Dropping {len(nodes_to_drop)} nodes: {nodes_to_drop}")
    
    # Remove selected nodes
    sub_graph.remove_nodes_from(nodes_to_drop)
    
    print(f"Original graph had {len(graph.nodes())} nodes, subgraph has {len(sub_graph.nodes())} nodes")
    
    return sub_graph

In [ ]:
# defender_subgraph = create_defender_subgraph(pre_sub_graph)
# defender_subgraphs_list = []
# defender_subgraphs_list.append(pre_sub_graph)

# create 3 different subgraphs and store them in a list
defender_subgraphs_list = [create_defender_subgraph(full_attack_graph) for _ in range(3)]

### Plot it

In [ ]:
def visualize_subgraphs(subgraph_list, original_graph=None):
    """
    Visualizes the last 3 defender's subgraphs side by side.
    """

    # Take only the last 3 graphs
    graphs_to_show = subgraph_list[-3:] if len(subgraph_list) >= 3 else subgraph_list
    num_graphs = len(graphs_to_show)
    
    # Create subplot figure
    fig, axes = plt.subplots(1, num_graphs, figsize=(6*num_graphs, 6))
    
    # Handle case when only one graph (axes not array)
    if num_graphs == 1:
        axes = [axes]
    
    for i, subgraph in enumerate(graphs_to_show):
        # If no original graph provided, use the subgraph
        orig_graph = original_graph if original_graph else subgraph
        
        # Identify original target nodes
        original_target_nodes = [n for n, d in orig_graph.out_degree() if d == 0]
        
        # Identify entry nodes in subgraph
        entry_nodes = [n for n, d in subgraph.in_degree() if d == 0]
        
        # Use spring layout
        pos = nx.spring_layout(subgraph, k=1, iterations=50, seed=42)
        
        # Draw edges with arrows
        nx.draw_networkx_edges(subgraph, pos, 
                              edge_color='gray',
                              arrows=True,
                              arrowsize=15,
                              width=1.5,
                              ax=axes[i])
        
        # Create color map for nodes
        node_colors = ['lightgreen' if node in entry_nodes else 
                      'lightcoral' if node in original_target_nodes else 
                      'lightblue' for node in subgraph.nodes()]
        
        # Draw nodes
        nx.draw_networkx_nodes(subgraph, pos,
                              node_color=node_colors,
                              node_size=500,
                              edgecolors='darkblue',
                              linewidths=1.5,
                              ax=axes[i])
        
        # Create and draw labels
        labels = {node: str(node) for node in subgraph.nodes()}
        nx.draw_networkx_labels(subgraph, pos,
                               labels,
                               font_size=10,
                               font_weight='bold',
                               ax=axes[i])
        
        axes[i].set_title(f"Defender's Subgraph {i+1}", fontsize=12, fontweight='bold')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_subgraphs(defender_subgraphs_list)

In [ ]:
print("create subsgraph for defender concluded")